<a href="https://colab.research.google.com/github/helmieresearch/helmieresearch/blob/main/Momentum_Ranking_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install packages

In [ ]:
!pip install yfinance

In [ ]:
!pip install yahooquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 1.3 MB/s eta 0:00:00


In [ ]:
!pip install stockstats

###Import data

In [ ]:
%matplotlib inline
import datetime
from datetime import date
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#import mpld3
#mpld3.enable_notebook()
"""
Settings
"""
portfolio_value = 2000.00
perf_last_month = 2000.00
start = "2020-01-01"
end = "2023-07-31"
ranking_day = "2023-07-28"

# Storing today's date into a variable
today = date.today()
# Storing the specific date
trading_day = today
# Calculate percent difference since last month
perf_pct = (portfolio_value / perf_last_month) - 1
# Print performance, format as percent with two decimals.
print("{} - Last Month Result: {:.2%}".format(trading_day, perf_pct))
# Remember today's portfolio value for next month's calculation
#perf_last_month = portfolio_value

2023-07-29 - Last Month Result: 0.00%


In [ ]:

now = datetime.datetime.now().strftime("%Y-%m-%d")
data = yf.Ticker("^J203.JO")
index = data.history(start=start,  end=end)
resetindex = index.reset_index(inplace = True)

index['Date'] = pd.to_datetime(index['Date']).dt.strftime('%Y-%m-%d')

index = index.drop(columns=(['Open', 'High','Low','Volume','Dividends','Stock Splits']))

index.set_index("Date", inplace=True)

index.to_csv('jse_index.csv')

In [ ]:
#Import required libraries
import pandas as pd
import json
import requests
import glob
import os
from yahooquery import Ticker
from datetime import datetime

params = {'access_key': 'e52cf3b93696352e880916f8c8adbf0c',
          'limit': 350}
api_result = requests.get('http://api.marketstack.com/v1/exchanges/XJSE/tickers', params)
api_response = api_result.json()
print(f"Exchange Name = {api_response['data']['name']}")
for ticker in api_response['data']['tickers']:
  print(f"{ticker['name']}: {ticker['symbol']}")


# Serializing json
json_object = json.dumps(api_response, indent=4)
# Writing to sample.json
with open("jse.json", "w") as outfile:
    outfile.write(json_object)

# Opening JSON file
with open('jse.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)
df1 = pd.json_normalize(json_object["data"]["tickers"])

# dropping null value columns to avoid errors
# new data frame with split value columns
new = df1["symbol"].str.split(".", n = 1, expand = True)
# making separate first name column from new data frame
df1["ticker"]= new[0]
# Dropping old Name columns
df1.drop(columns =["has_intraday","has_eod"], inplace = True)
new.columns = new.columns.astype(str)
new.columns.values[1] = "tickers"
new['0'] + '.' + new['tickers']
new.replace("XJSE", "JO", inplace=True)

result = pd.concat([df1, new], axis=1, join='inner')
df2=result['0'] + '.' + result['tickers']
final = pd.concat([df1, df2], axis=1, join='inner')
final.columns.values[3] = "tickers"
final.columns = final.columns.astype(str)
#Dropping old Name columns
final.drop(columns =["name", "symbol","ticker"], inplace = True)
# Converting a specific Dataframe
# column to list using Series.tolist()
xjse = final["tickers"].tolist()
print("Converting tickers to list...")
print("Output dataframe..." )
print("Writing csv file..." )
print("Saving JSE data to disc...")
print("Saving csv files for each stock to disc...")
symbols = xjse
tickers = Ticker(symbols, asynchronous=True)
data = tickers.history(start= start, end=end, interval='1d')
data= data.drop(['adjclose','dividends','splits'],axis=1)


#Convert dataframe to csv and save to disc
final.to_csv("tickers.csv")
data.to_csv("jse_stocks.csv")

resetindex = data.reset_index(inplace = True)

for i, g in data.groupby('symbol'):
    g.to_csv('/content/Stocks/{}.csv'.format(i), header=True, index_label=False)

# merging the files
joined_files = os.path.join("/content/Stocks/", "*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# new dataframe with different column order
df_new = df.iloc[:, [1, 2, 3, 4, 5, 6, 0]]

Tickers = df_new["symbol"].tolist()
mylist = list(dict.fromkeys(Tickers))
df = pd.DataFrame (mylist, columns = ['tickers'])
df.to_csv("/content/tickers.csv")

#Change date format to Year-Month-Day
df_new['date'] =  pd.to_datetime(df_new['date']).dt.strftime('%Y-%m-%d')

df_new.set_index('date', inplace=True)

for i, g in df_new.groupby('symbol'):
    g.to_csv('/content/Stocks_final3/{}.csv'.format(i), header=True, index_label=False)

Exchange Name = Johannesburg Stock Exchange
PROSUS N.V.: PRX.XJSE
ANHEUSER-BUSCH INBEV SA NV: ANH.XJSE
BHP GROUP PLC: BHP.XJSE
BRITISH AMERICAN TOB PLC: BTI.XJSE
NASPERS LTD -N-: NPN.XJSE
COMPAGNIE FIN RICHEMONT: CFR.XJSE
GLENCORE PLC: GLN.XJSE
ANGLO AMERICAN PLC: AGL.XJSE
FIRSTRAND LTD: FSR.XJSE
ANGLO AMERICAN PLAT LTD: AMS.XJSE
STANDARD BANK GROUP LTD: SBK.XJSE
VODACOM GROUP LTD: VOD.XJSE
MONDI PLC: MNP.XJSE
CAPITEC BANK HLDGS LTD: CPI.XJSE
BEE - SASOL LIMITED: SOLBE1.XJSE
SANLAM LIMITED: SLM.XJSE
MTN GROUP LTD: MTN.XJSE
ANGLOGOLD ASHANTI LTD: ANG.XJSE
SASOL LIMITED: SOL.XJSE
ABSA GROUP LIMITED: ABG.XJSE
SOUTH32 LIMITED: S32.XJSE
RMB HOLDINGS LTD: RMH.XJSE
IMPALA PLATINUM HLGS LTD: IMP.XJSE
BID CORPORATION LTD: BID.XJSE
REMGRO LTD: REM.XJSE
KUMBA IRON ORE LTD: KIO.XJSE
NEDBANK GROUP LTD: NED.XJSE
SIBANYE STILLWATER LTD: SSW.XJSE
GOLD FIELDS LTD: GFI.XJSE
INVESTEC LTD: INL.XJSE
INVESTEC PLC: INP.XJSE
OLD MUTUAL LIMITED: OMU.XJSE
NEPI ROCKCASTLE PLC: NRP.XJSE
DISCOVERY LTD: DSY.XJSE
SH

In [ ]:
symbols = xjse
tickers = Ticker(symbols, asynchronous=True)
data = tickers.history(start=start, end=end, interval='1d')
data= data.drop(['adjclose','dividends','splits'],axis=1)

resetindex = data.reset_index(inplace = True)

# new dataframe with different column order
df_new = data.iloc[:, [1, 2, 3, 4, 5, 6, 0]]

Tickers = df_new["symbol"].tolist()
mylist = list(dict.fromkeys(Tickers))
df = pd.DataFrame (mylist, columns = ['tickers'])
df.columns = df.iloc[1]

df.to_csv("/content/Stocks_final3/tickers.csv")

from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats

csv_folder = Path('/content/Stocks_final3')

for file in csv_folder.glob('*.JO.csv'): #create iteratable & iterate on it.
    mydata = pd.read_csv(file)
    mydata.reset_index(inplace = True)
    mydata = mydata.rename(columns={'index': 'date'}, index=None)
    mydata['date'] =  pd.to_datetime(mydata['date']).dt.strftime('%Y-%m-%d')
    mydata.set_index('date', inplace=True)
    df =mydata[["open","high","low","close","volume"]]
    no_outlier_prices = df[(np.abs(stats.zscore(df)) <3).all(axis=1)]
    new_file_name = file.parent.joinpath(f"{file.stem}.csv")
    no_outlier_prices.to_csv(new_file_name) #, index=None, encoding='utf-8-sig', decimal=',')

###Ranking list to construct initial portfolio

In [ ]:
%matplotlib inline
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (12, 9) # (w, h)
plt.ioff()

tickers = pd.read_csv('/content/Stocks_final3/tickers.csv', header=None)[1].tolist()
stocks = (
    (pd.concat(
        [pd.read_csv(f"/content/Stocks_final3/{ticker}.csv",parse_dates=None, index_col=0)[
        'close'].rename(ticker)
        for ticker in tickers],
        axis=1,
        sort=True)
    )
)
stocks = stocks.loc[:,~stocks.columns.duplicated()]

from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2

momentums = stocks.copy(deep=True)
for ticker in tickers:
    momentums[ticker] = stocks[ticker].rolling(90).apply(momentum, raw=False)

result = momentums.sort_values(by =ranking_day, axis = 1, ascending =False)
result.reset_index(inplace=True)
res=result[~(result['date'] < ranking_day)].dropna(axis=1)

mes = res.reindex(columns = res.columns.tolist()
                                  + ['empty'])

df = mes.set_index(['empty']).stack().reset_index(name='index').rename(columns={'level_2':'date'})

df.columns = df.iloc[0]

df.index = list(df.index)

df.columns.values[0] = "empty"
df.columns.values[1] = "symbol"
df.columns.values[2] = "momentum_score"

df=df.drop(['empty'], axis=1)

finalframe=df.drop(index=0)

finalframe['momentum_score'] =finalframe['momentum_score'].astype(float)

finalframe['rank']=finalframe['momentum_score'].rank(ascending=False)

finalframe['momentum_score'] =finalframe['momentum_score'].astype(float)

#finalframe.style.format({'momentum_score':"{:.2f}"})

Mylist = finalframe["symbol"].tolist()

finalframe.set_index('symbol',inplace=True)

final = pd.concat([df1, df2], axis=1, join='inner')
final.columns.values[3] = "tickers"
final.columns = final.columns.astype(str)

final.set_index('tickers', inplace=True)
Yes = final.reindex(Mylist)

rt = pd.concat([finalframe, Yes], axis=1, join='inner')

rt = rt.iloc[:, [2, 3, 4, 0, 1]]

#rt.style.format({'momentum_score':"{:.2f}"})

# get names of indexes for which column "momentum_score" has value > 40%
index_names = rt[(rt['momentum_score'] < 0.001)].index

# drop these given row indexes from dataFrame
rt.drop(index_names, inplace = True)

rt.reset_index(inplace=True)

My_final_list = rt["index"].tolist()

rt.set_index('index',inplace=True)

final_table_columns = My_final_list
final_table =stocks.drop(columns=[col for col in stocks if col not in final_table_columns])

for column in final_table.columns:
    final_table[column].to_csv('/content/momentum_tickers/' + column + '.csv')

In [ ]:
from pathlib import Path
import pandas as pd
import stockstats
from stockstats import StockDataFrame as sdf
import glob
vola_table = Path('/content/momentum_tickers')

for file in vola_table.glob('*.JO.csv'): #create iteratable & iterate on it.
    vola_df = pd.read_csv(file)
    vola_df.columns.values[1] = "close"
    vola_df["20d_vol"] = vola_df["close"].pct_change().rolling(20).std()
    #vola_df = sdf.retype(vola_df)['close_90_roc']
    #vola_df = pd.DataFrame(vola_df)
    vola_df['date'] =  pd.to_datetime(vola_df['date']).dt.strftime('%Y-%m-%d')
    vola_df.set_index('date', inplace=True)
    stockstats_df = sdf.retype(vola_df)['close_90_roc']
    df = pd.DataFrame(stockstats_df)
    vola_file_name= file.parent.joinpath(f"/content/ranking_table/{file.stem}.csv")
    vola_df.to_csv(vola_file_name) #, index=None, encoding='utf-8-sig', decimal=',')

vola_table1 = Path('/content/ranking_table')

for file in vola_table1.glob('*.JO.csv'): #create iteratable & iterate on it.
    vola_df1 = pd.read_csv(file).tail(1)

    vola_df1['date'] =  pd.to_datetime(vola_df1['date']).dt.strftime('%Y-%m-%d')

    vola_df1.set_index('date', inplace=True)

    vola_file_name1= file.parent.joinpath(f"/content/momentum_stocks/{file.stem}.csv")

    vola_df1.to_csv(vola_file_name1)


# set search path and glob for files
# here we want to look for csv files in the input directory
path = '/content/'
files = glob.glob(path + '/momentum_stocks/*.csv')

# create empty list to store dataframes
li = []

# loop through list of files and read each one into a dataframe and append to list
for f in files:
    # get filename
    stock = os.path.basename(f)
    # read in csv
    temp_df = pd.read_csv(f)
    # create new column with filename
    temp_df['ticker'] = stock
    # data cleaning to remove the .csv
    temp_df['ticker'] = temp_df['ticker'].replace('.csv', '', regex=True)
    # append df to list
    li.append(temp_df)
    #print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

# concatenate our list of dataframes into one!
df = pd.concat(li)

ranking_list =df.set_index('ticker').reindex(My_final_list).reset_index()

inv_vol = 1 / ranking_list["20d_vol"]
sum_inv_vol=inv_vol.sum()
vol_targets = inv_vol / sum_inv_vol
ranking_list["weight"] = vol_targets

ranking_list.set_index('ticker', inplace=True)

final_list = pd.concat([rt,ranking_list], axis=1, join='inner')

stockprice = final_list["close"] / 100
pos_weight = portfolio_value * final_list["weight"]
final_list["units"] = pos_weight / stockprice
final_list["ZAR"] = stockprice * final_list["units"]

In [ ]:
from pathlib import Path
import pandas as pd

sma = Path('/content/ranking_table')

for file in sma.glob('*.JO.csv'): #create iteratable & iterate on it.
    sma = pd.read_csv(file)
    sma['date'] =  pd.to_datetime(sma['date']).dt.strftime('%Y-%m-%d')
    sma.set_index('date', inplace=True)

    '''Function'''

    sma['100d_sma'] = sma['close'].rolling(100).mean()

    #sma['index_sma']=index['Close'].rolling(200).mean()

    #sma['trend'] = np.where(sma['close'] > sma['100d_sma'], True, False)

    sma['index'] = index['Close']

    sma['index_sma']=index['Close'].rolling(200).mean()

    sma['trend'] = np.where(sma['close'] > sma['100d_sma'], True, False)

    sma['index_trend']= np.where(sma['index'] > sma['index_sma'], True, False)

    sma_file_name= file.parent.joinpath(f"/content/sma_data/{file.stem}.csv")

    sma.to_csv(sma_file_name)

buy_table = Path('/content/sma_data')

for file in buy_table.glob('*.JO.csv'): #create iteratable & iterate on it.
    buy_list_df = pd.read_csv(file).tail(1)

    buytable = file.parent.joinpath(f"/content/buy_list/{file.stem}.csv")

    buy_list_df.to_csv(buytable)

In [ ]:
import pandas as pd
import glob

# set search path and glob for files
# here we want to look for csv files in the input directory
path1 = '/content/'
files1 = glob.glob(path1 + '/buy_list/*.csv')

# create empty list to store dataframes
li1 = []

# loop through list of files and read each one into a dataframe and append to list
for f1 in files1:
    # get filename
    stock1 = os.path.basename(f1)
    # read in csv
    temp_df1 = pd.read_csv(f1)
    # create new column with filename
    temp_df1['ticker'] = stock1
    # data cleaning to remove the .csv
    temp_df1['ticker'] = temp_df1['ticker'].replace('.csv', '', regex=True)
    # append df to list
    li1.append(temp_df1)
    #print(f'Successfully created dataframe for {stock1} with shape {temp_df1.shape}')

# concatenate our list of dataframes into one!
Buy = pd.concat(li1)

buying_list = Buy.set_index('ticker').reindex(My_final_list)#.reset_index()

final_buy_list = pd.concat([final_list, buying_list], axis=1, join='inner')

#define list of columns range
cols = [x for x in range(final_buy_list.shape[1])]

#remove second column in DataFrame
cols.remove(12)
cols.remove(13)
cols.remove(14)
cols.remove(15)
cols.remove(16)#view resulting DataFrame
final_buy_list2 =final_buy_list.iloc[:, cols]
final_buy_list2.info()

final_buy_list2 = final_buy_list2.loc[final_buy_list2.trend, :]

index_names1 = final_buy_list2[(final_buy_list2['close_90_roc'] >= 15)].index

# drop these given row indexes from dataFrame
final_buy_list2.drop(index_names1, inplace = True)
final_buy_list2.to_csv("portfolio_280723.csv")
final_buy_list2

<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, TPF.JO to CAT.JO
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            68 non-null     object 
 1   symbol          68 non-null     object 
 2   ticker          68 non-null     object 
 3   momentum_score  68 non-null     float64
 4   rank            68 non-null     float64
 5   date            68 non-null     object 
 6   close           68 non-null     float64
 7   20d_vol         68 non-null     float64
 8   close_90_roc    67 non-null     float64
 9   weight          68 non-null     float64
 10  units           68 non-null     float64
 11  ZAR             68 non-null     float64
 12  100d_sma        64 non-null     float64
 13  index           68 non-null     float64
 14  index_sma       68 non-null     float64
 15  trend           68 non-null     bool   
 16  index_trend     68 non-null     bool   
dtypes: bool(2), float64(11), object(4

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
TPF.JO,TRANSCEND RES PROP FD LD,TPF.XJSE,TPF,147.704351,1.0,2023-07-28,639.0,22.149794,-0.776398,0.000012,0.003754,0.023987,446.5455,78506.898438,75498.789805,True,True
SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.098268,5.0,2023-07-28,1882.0,0.014916,11.492891,0.017811,1.892749,35.621534,1878.8200,78506.898438,75498.789805,True,True
INL.JO,INVESTEC LTD,INL.XJSE,INL,1.015240,6.0,2023-07-28,11150.0,0.014252,10.857029,0.018640,0.334351,37.280139,10422.1900,78506.898438,75498.789805,True,True
ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.948768,7.0,2023-07-28,1590.0,0.016859,6.425703,0.015757,1.982060,31.514752,1514.1700,78506.898438,75498.789805,True,True
INP.JO,INVESTEC PLC,INP.XJSE,INP,0.942281,8.0,2023-07-28,11201.0,0.014045,10.354680,0.018915,0.337734,37.829588,10510.7600,78506.898438,75498.789805,True,True
QFH.JO,QUANTUM FOODS HLDGS LTD,QFH.XJSE,QFH,0.584236,9.0,2023-07-28,480.0,0.025495,6.666667,0.010420,4.341605,20.839705,454.6900,78506.898438,75498.789805,True,True
CLI.JO,CLIENTELE LTD,CLI.XJSE,CLI,0.578104,10.0,2023-07-28,1190.0,0.052228,13.333333,0.005087,0.854875,10.173012,1059.6000,78506.898438,75498.789805,True,True
TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.569492,12.0,2023-07-28,653.0,0.018276,8.471761,0.014536,4.452021,29.071695,647.1500,78506.898438,75498.789805,True,True
LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.427845,16.0,2023-07-28,2104.0,0.008620,11.381683,0.030819,2.929573,61.638216,2002.2500,78506.898438,75498.789805,True,True
DCP.JO,DIS-CHEM PHARMACIES LTD,DCP.XJSE,DCP,0.321957,26.0,2023-07-28,2570.0,0.022596,4.301948,0.011757,0.914944,23.514054,2464.9300,78506.898438,75498.789805,True,True


###List to Liquidate current positions

In [ ]:
#read current portfolio from disc
sell=pd.read_csv("portfolio_280723.csv")
sell
'''
#save tickers of current poistions
#to list
pf = sell["Unnamed: 0"].tolist()
pf
'''

sell_list = final_buy_list [~final_buy_list
 ["trend"]]
sell_list.info()
sell_list

#sell = pf

#sell_list[sell_list['index'].isin(sell)]

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, RBP.JO to CAT.JO
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            36 non-null     object 
 1   symbol          36 non-null     object 
 2   ticker          36 non-null     object 
 3   momentum_score  36 non-null     float64
 4   rank            36 non-null     float64
 5   date            36 non-null     object 
 6   close           36 non-null     float64
 7   20d_vol         36 non-null     float64
 8   close_90_roc    35 non-null     float64
 9   weight          36 non-null     float64
 10  units           36 non-null     float64
 11  ZAR             36 non-null     float64
 12  Unnamed: 0      36 non-null     int64  
 13  date            36 non-null     object 
 14  close           36 non-null     float64
 15  20d_vol         36 non-null     float64
 16  close_90_roc    35 non-null     float64
 17  100d_sma        32 non-null     f

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,...,Unnamed: 0,date,close,20d_vol,close_90_roc,100d_sma,index,index_sma,trend,index_trend
RBP.JO,ROYAL BAFOKENG PLATINUM LTD,RBP.XJSE,RBP,0.505269,13.0,2023-07-28,12785.0,0.014911,-11.516368,0.017816,...,894,2023-07-28,12785.0,0.014911,-11.516368,13833.2700,78506.898438,75498.789805,False,True
JSE.JO,JSE LTD,JSE.XJSE,JSE,0.487616,14.0,2023-07-28,9115.0,0.010982,NaN,0.024190,...,894,2023-07-28,9115.0,0.010982,NaN,NaN,78506.898438,75498.789805,False,True
BWN.JO,BALWIN PROPERTIES LTD,BWN.XJSE,BWN,0.470207,15.0,2023-07-28,250.0,0.008892,-18.566775,0.029875,...,894,2023-07-28,250.0,0.008892,-18.566775,280.8700,78506.898438,75498.789805,False,True
ARI.JO,AFRICAN RAINBOW MIN LTD,ARI.XJSE,ARI,0.417842,17.0,2023-07-28,19748.0,0.018049,-11.483640,0.014718,...,894,2023-07-28,19748.0,0.018049,-11.483640,22090.1800,78506.898438,75498.789805,False,True
GPL.JO,GRAND PARADE INV LTD,GPL.XJSE,GPL,0.381187,18.0,2023-07-28,293.0,0.022673,-9.846154,0.011717,...,894,2023-07-28,293.0,0.022673,-9.846154,318.2100,78506.898438,75498.789805,False,True
YRK.JO,YORK TIMBER HOLDINGS LTD,YRK.XJSE,YRK,0.378071,19.0,2023-07-28,189.0,0.029677,-14.479638,0.008951,...,894,2023-07-28,189.0,0.029677,-14.479638,203.0800,78506.898438,75498.789805,False,True
RMH.JO,RMB HOLDINGS LTD,RMH.XJSE,RMH,0.368895,20.0,2023-07-28,48.0,0.014148,-7.692308,0.018777,...,894,2023-07-28,48.0,0.014148,-7.692308,49.9600,78506.898438,75498.789805,False,True
KAL.JO,KAAP AGRI LIMITED,KAL.XJSE,KAL,0.366622,21.0,2023-07-28,3717.0,0.010306,-6.842105,0.025777,...,894,2023-07-28,3717.0,0.010306,-6.842105,3806.9400,78506.898438,75498.789805,False,True
TBS.JO,TIGER BRANDS LTD,TBS.XJSE,TBS,0.336562,22.0,2023-07-28,16080.0,0.017263,-14.354194,0.015389,...,894,2023-07-28,16080.0,0.017263,-14.354194,18207.4200,78506.898438,75498.789805,False,True
MPT.JO,MPACT LIMITED,MPT.XJSE,MPT,0.334141,23.0,2023-07-28,2663.0,0.015813,-10.697518,0.016800,...,894,2023-07-28,2663.0,0.015813,-10.697518,2717.8100,78506.898438,75498.789805,False,True


In [ ]:
dqs = sell_list ["ticker"].tolist()
dqs

['RBP',
 'JSE',
 'BWN',
 'ARI',
 'GPL',
 'YRK',
 'RMH',
 'KAL',
 'TBS',
 'MPT',
 'WEZ',
 'HLM',
 'APF',
 'REN',
 'LEW',
 'CMH',
 'NTC',
 'DRD',
 'GML',
 'GSH',
 'PPE',
 'LAB',
 'AEG',
 'SOH',
 'IVT',
 'PPC',
 'PBG',
 'MFL',
 'MST',
 'GNDP',
 'ATT',
 'MTNZF',
 'SDO',
 'JBL',
 'QLT',
 'CAT']

In [34]:
cpf=pd.read_csv("portfolio_140723.csv")
cpf.info()
cpf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      27 non-null     object 
 1   name            27 non-null     object 
 2   symbol          27 non-null     object 
 3   ticker          27 non-null     object 
 4   momentum_score  27 non-null     float64
 5   rank            27 non-null     float64
 6   date            27 non-null     object 
 7   close           27 non-null     float64
 8   20d_vol         27 non-null     float64
 9   close_90_roc    27 non-null     float64
 10  weight          27 non-null     float64
 11  units           27 non-null     float64
 12  ZAR             27 non-null     float64
 13  100d_sma        27 non-null     float64
 14  index           27 non-null     float64
 15  index_sma       27 non-null     float64
 16  trend           27 non-null     bool   
 17  index_trend     27 non-null     bool 

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,VUN.JO,VUNANI LTD,VUN.XJSE,VUN,59.892215,1.0,2023-07-14,260.0,0.018176,-10.344828,0.013319,10.245601,26.638563,236.1500,77750.507812,74859.902324,True,True
1,SEB.JO,SEBATA HOLDINGS LTD,SEB.XJSE,SEB,3.075193,2.0,2023-07-14,220.0,0.109355,-12.000000,0.002214,2.012527,4.427560,188.4971,77750.507812,74859.902324,True,True
2,SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.544453,4.0,2023-07-14,1972.0,0.019070,9.799555,0.012695,1.287476,25.389029,1870.5900,77750.507812,74859.902324,True,True
3,TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.923557,7.0,2023-07-14,670.0,0.035485,8.064516,0.006822,2.036480,13.644417,641.8500,77750.507812,74859.902324,True,True
4,ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.613898,8.0,2023-07-14,1632.0,0.012478,9.898990,0.019402,2.377644,38.803154,1502.8500,77750.507812,74859.902324,True,True
5,REM.JO,REMGRO LTD,REM.XJSE,REM,0.606838,9.0,2023-07-14,15300.0,0.011028,10.365722,0.021952,0.286955,43.904071,13924.5900,77750.507812,74859.902324,True,True
6,LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.439935,16.0,2023-07-14,2049.0,0.010084,5.673027,0.024007,2.343246,48.013120,1989.7600,77750.507812,74859.902324,True,True
7,RLO.JO,REUNERT LTD,RLO.XJSE,RLO,0.274446,34.0,2023-07-14,5947.0,0.015125,0.337439,0.016006,0.538298,32.012604,5914.1000,77750.507812,74859.902324,True,True
8,ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,ABG,0.271652,35.0,2023-07-14,17804.0,0.012469,-10.420126,0.019416,0.218105,38.831346,17379.3800,77750.507812,74859.902324,True,True
9,PPH.JO,PEPKOR HOLDINGS LTD,PPH.XJSE,PPH,0.232722,37.0,2023-07-14,1695.0,0.016413,-6.714364,0.014750,1.740358,29.499060,1679.2700,77750.507812,74859.902324,True,True


In [30]:
liquidate = cpf.loc[cpf['ticker'].isin (dqs)].reset_index (drop=True)
liquidate

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,MFL.JO,METROFILE HOLDINGS LTD,MFL.XJSE,MFL,0.039159,63.0,2023-07-14,325.0,0.066643,5.177994,0.003633,2.235468,7.265271,313.42,77750.507812,74859.902324,True,True
1,JBL.JO,JUBILEE METALS GROUP PLC,JBL.XJSE,JBL,0.015929,70.0,2023-07-14,193.0,0.054717,-8.530806,0.004424,4.584854,8.848768,191.44,77750.507812,74859.902324,True,True


In [36]:
newpf = cpf[~cpf['ticker'].isin(liquidate['ticker'])]
newpf .info()
newpf

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 26
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      25 non-null     object 
 1   name            25 non-null     object 
 2   symbol          25 non-null     object 
 3   ticker          25 non-null     object 
 4   momentum_score  25 non-null     float64
 5   rank            25 non-null     float64
 6   date            25 non-null     object 
 7   close           25 non-null     float64
 8   20d_vol         25 non-null     float64
 9   close_90_roc    25 non-null     float64
 10  weight          25 non-null     float64
 11  units           25 non-null     float64
 12  ZAR             25 non-null     float64
 13  100d_sma        25 non-null     float64
 14  index           25 non-null     float64
 15  index_sma       25 non-null     float64
 16  trend           25 non-null     bool   
 17  index_trend     25 non-null     bool 

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,VUN.JO,VUNANI LTD,VUN.XJSE,VUN,59.892215,1.0,2023-07-14,260.0,0.018176,-10.344828,0.013319,10.245601,26.638563,236.1500,77750.507812,74859.902324,True,True
1,SEB.JO,SEBATA HOLDINGS LTD,SEB.XJSE,SEB,3.075193,2.0,2023-07-14,220.0,0.109355,-12.000000,0.002214,2.012527,4.427560,188.4971,77750.507812,74859.902324,True,True
2,SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.544453,4.0,2023-07-14,1972.0,0.019070,9.799555,0.012695,1.287476,25.389029,1870.5900,77750.507812,74859.902324,True,True
3,TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.923557,7.0,2023-07-14,670.0,0.035485,8.064516,0.006822,2.036480,13.644417,641.8500,77750.507812,74859.902324,True,True
4,ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.613898,8.0,2023-07-14,1632.0,0.012478,9.898990,0.019402,2.377644,38.803154,1502.8500,77750.507812,74859.902324,True,True
5,REM.JO,REMGRO LTD,REM.XJSE,REM,0.606838,9.0,2023-07-14,15300.0,0.011028,10.365722,0.021952,0.286955,43.904071,13924.5900,77750.507812,74859.902324,True,True
6,LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.439935,16.0,2023-07-14,2049.0,0.010084,5.673027,0.024007,2.343246,48.013120,1989.7600,77750.507812,74859.902324,True,True
7,RLO.JO,REUNERT LTD,RLO.XJSE,RLO,0.274446,34.0,2023-07-14,5947.0,0.015125,0.337439,0.016006,0.538298,32.012604,5914.1000,77750.507812,74859.902324,True,True
8,ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,ABG,0.271652,35.0,2023-07-14,17804.0,0.012469,-10.420126,0.019416,0.218105,38.831346,17379.3800,77750.507812,74859.902324,True,True
9,PPH.JO,PEPKOR HOLDINGS LTD,PPH.XJSE,PPH,0.232722,37.0,2023-07-14,1695.0,0.016413,-6.714364,0.014750,1.740358,29.499060,1679.2700,77750.507812,74859.902324,True,True


In [35]:
final_buy_list2

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
TPF.JO,TRANSCEND RES PROP FD LD,TPF.XJSE,TPF,147.704351,1.0,2023-07-28,639.0,22.149794,-0.776398,0.000012,0.003754,0.023987,446.5455,78506.898438,75498.789805,True,True
SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.098268,5.0,2023-07-28,1882.0,0.014916,11.492891,0.017811,1.892749,35.621534,1878.8200,78506.898438,75498.789805,True,True
INL.JO,INVESTEC LTD,INL.XJSE,INL,1.015240,6.0,2023-07-28,11150.0,0.014252,10.857029,0.018640,0.334351,37.280139,10422.1900,78506.898438,75498.789805,True,True
ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.948768,7.0,2023-07-28,1590.0,0.016859,6.425703,0.015757,1.982060,31.514752,1514.1700,78506.898438,75498.789805,True,True
INP.JO,INVESTEC PLC,INP.XJSE,INP,0.942281,8.0,2023-07-28,11201.0,0.014045,10.354680,0.018915,0.337734,37.829588,10510.7600,78506.898438,75498.789805,True,True
QFH.JO,QUANTUM FOODS HLDGS LTD,QFH.XJSE,QFH,0.584236,9.0,2023-07-28,480.0,0.025495,6.666667,0.010420,4.341605,20.839705,454.6900,78506.898438,75498.789805,True,True
CLI.JO,CLIENTELE LTD,CLI.XJSE,CLI,0.578104,10.0,2023-07-28,1190.0,0.052228,13.333333,0.005087,0.854875,10.173012,1059.6000,78506.898438,75498.789805,True,True
TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.569492,12.0,2023-07-28,653.0,0.018276,8.471761,0.014536,4.452021,29.071695,647.1500,78506.898438,75498.789805,True,True
LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.427845,16.0,2023-07-28,2104.0,0.008620,11.381683,0.030819,2.929573,61.638216,2002.2500,78506.898438,75498.789805,True,True
DCP.JO,DIS-CHEM PHARMACIES LTD,DCP.XJSE,DCP,0.321957,26.0,2023-07-28,2570.0,0.022596,4.301948,0.011757,0.914944,23.514054,2464.9300,78506.898438,75498.789805,True,True


In [37]:
musthave = final_buy_list2 ["ticker"].tolist()
musthave

['TPF',
 'SRE',
 'INL',
 'ART',
 'INP',
 'QFH',
 'CLI',
 'TRE',
 'LHC',
 'DCP',
 'RDF',
 'MTH',
 'ASC',
 'SPG',
 'PPH',
 'MTM',
 'MSP',
 'ABG',
 'NPN',
 'PRX',
 'SUR',
 'ARL',
 'NWL',
 'HYP']